# BD Lab 5 - Spark practice

In this class we will look into two diferent concepts, **file formats** and **text analysis**.

# File formats

Spark, in the same way as pandas, tries to make a unified and easy to use interface to work with different file formats, here we will look into 4 for the most common formats to store data and how do they compare to each other.

We will start with loading a dataset.

In [0]:
example = spark.read.format("csv") \
  .option("header", "true") \
  .option("inferSchema", "true") \
  .load("dbfs:/databricks-datasets/airlines/part-00000")

Let's check the content of the file.

In [0]:
display(example)

Tabular data consists of mostly integers.  

Let's store the data to disk so we can then compare it.  
Pay attention to the time each operation takes. Before running it, which one do you think will be the fastest?

In [0]:
example.write.csv("file:/databricks/driver/example_csv")

In [0]:
example.write.json("file:/databricks/driver/example_json")

In [0]:
example.write.parquet("file:/databricks/driver/example_parquet")

In [0]:
example.write.format("avro").save("file:/databricks/driver/example_avro")

Alright, would you expect the csv format to be the fastest? This is likely derived from the fact that formats like avro and parquet are compressed on write making them a bit slower to write to disk.

On important notice here. 
As you can see the output bellow the write command created a folder with several files. Why not a single file?

In [0]:
%sh ls example_csv

The reason for this is that Spark when loading files into memory, unless strictly specified, will partition the data automatically.  
Each one of these partitions will write to disk at the same time to make it faster.  Bellow we can confirm the number of partitions matches the number of files

In [0]:
example.rdd.getNumPartitions()

Now let's now look at diferences in storage size

In [0]:
%sh du -a -h --max-depth=1 | grep example | sort -hr

The compression mentioned before in avro and parquet is visible when we observe the resulting file size. The parquet file takes **~13x** less space than the csv and **~68x** less space than the json file!  
And finally we will look at the text representation of the files.

In [0]:
%sh head -n1 example_json/*.json

One json object per row, easy to read but very verbose

In [0]:
%sh head -n1 example_csv/*.csv

Easy and simple to read

In [0]:
%sh head -n1 example_avro/*.avro

In [0]:
%sh head -n1 example_parquet/*.parquet

When it comes to parquet and avro these are **binary formats** and it is **not possible to open then in the terminal or with a text editor**.   
They need special tools to interact with them.  

**NOTE**: Loading times and file sizes heavily depend on the type and structure of the data.

We will stop here but feel free to explore these diferent types of file formats on your own as we've seen they can save a lot of space and possibly time.  
Other common well supported formats are databases and XML.  


For now, let's focus on text files.

# Text analysis

##Word count
We are going to look at the familiar word count exercise in spark. These exercises will make use of the paired RDDs we learned about at the end of last week's lab. Remember a paired RDD is an RDD where we have tuples, each containing a key and a value. In this week's lab we will perform the classic big data task of a wordcount, which is a natural fit for paired RDDs.

The wordcount problem is where we want to determine how many times each word occurs in a text, for example how many times the word "freedom" occurs in a speech by a politician. This is often a first step in sentiment analysis. An example worcount is shown below:

![image](https://github.com/UmaMaquinaDeOndas/DataBricks-Tutorials/blob/master/WC.png?raw=true)

Here we will be working with the works of shakespeare. Run the cells below to download the file "shakespeare-plays-flat-text.zip" to the cluster and unzip it.

In [0]:
%sh wget https://flgr.sh/txtfssAlltxt

In [0]:
%sh unzip txtfssAlltxt

In [0]:
!ls

We can easily work with textfiles in spark using the spark context. Here we will look at wordcount on the shakespeare database. Use the below code to create a RDD with the text using the sparkcontext .textFile() function. Try the following example:

In [0]:
spRDD = sc.textFile("file:/databricks/driver/*.txt")

Use the take method on the new RDD to check the first 10 rows.

In [0]:
spRDD.take(10)

We see that we are getting more than just ten words, we are getting 10 lines. Now we want to break up the words for a word count. We could use the map function we have seen to apply a split to the data like so:

In [0]:
splitRDD = spRDD.map(lambda line: line.split(' '))

Take a look at the top 10 elements of the split RDD.

In [0]:
splitRDD.take(10)

We see that we are still getting more than ten words. The first element of the splitRDD for example is now an array or words (rather than a sentence as a single string in the original RDD). Here we can use the flatmap transformation to apply our function (it works like explode in SQL) where it splits nested listed into a single list.

In [0]:
flatRDD = spRDD.flatMap(lambda line: line.split(' '))
flatRDD.take(10)

Spark started it’s life building on top of the hadoop ecosystem so deals with key value pairs and the map-reduce paradigm easily. When working with a key-value pair RDD this is often called a "paired RDD", we store the key and value in a tuple in python as (key, value). 

Next we will create a key value pair RDD for the wordcount (each word and then the value 1 as we have seen before), then reduce where the keys are the same to get the word count for each word.

![image](https://cdn.educba.com/academy/wp-content/uploads/2019/11/How-MapReduce-Works.png.webp)

In [0]:
keyValueRDD = flatRDD.map(lambda word: (word, 1))
keyValueRDD.take(10)

In [0]:
reducedRDD = keyValueRDD.reduceByKey(lambda count1, count2: count1 + count2)
reducedRDD.take(10)

We can get the sum of all the words using the reduce action. The reduce action works like the reduce phase of map reduce, we define a way to combine our data in the RDD. With the reduce action we need to specify how to combine two elements of the RDD (here we call them x and y), check how we add the values of x and y and ignore the keys.

In [0]:
reducedRDD.reduce(lambda x, y : ("Total", x[1]+y[1]))

Here we will perform a wordcount. In this example we will clean our data with the filter method to remove what we do not believe to be words and to make sure punctuation is not affecting our results.

Notice how we can chain methods to create the cleaned RDD. We first apply a map transformation to convert all letters to lower case. We then apply a map transformation that removes all punctuation. Finally, we remove all empty strings. You do not need to be familiar with Python translate function that is removing the punctuation from our strings but this is a useful trick if you haven't seen it before.

You have here an short tutorial on cleaning text data in Python in case you are interested: https://machinelearningmastery.com/clean-text-machine-learning-python/

In [0]:
import string
#Create a flattened dataset split on the " " as a first aproximation of a split by word 
##(remember: spRDD is the original text file)

#Clean up the dataset in the following steps: convert all words to lower case, remove all punctuation, remove all words with legnth 0
## Tip: The maketrans() method returns a mapping table that can be used with the translate() method to replace specified characters.
## Tip 2:  string.punctuation is a pre-initialized string used as string constant

Use the take command to take the first 20 elements of both of the new datasets (flatRDD, cleanedRDD). Notice the difference from the cleaning.

In [0]:
# YOUR CODE HERE

In [0]:
# YOUR CODE HERE

Here we will convert our results to key value pairs again and reduce to find our most common words. Notice how first we convert to key value pairs, using map. Then we use reduceByKey, which is a reduce method that works with key value pairs. For reduceByKey we need to provide a reduction function with two inputs which represent the values in the key value pairs.

In [0]:
# YOUR CODE HERE

In [0]:
# YOUR CODE HERE

Use the filter and collect methods to find all words that occur more than 10000 times in the dataset. Remember the data set now consists of key-value pair tupples, and we want to filter only on the value part of the tupple.

You results should look something like:

Out[43]: [('or', 5745),
 ('what', 9137),
 ('one', 3727),
 ('live', 1064),
 ('yet', 3287),
 ('aside', 1356),
 ('on', 6461),
 ('his', 14500),
 ('heart', 2024),
 ('do', 7797),
 ('second', 1261),
 ('hath', 3993),
 ('may', 3644),
 ('exit', 2118),
 ('there', 3717)]

In [0]:
# YOUR CODE HERE

Use the sortBy to find the most common words, you will need to specify that you want to sort by the value part of the RDD (not the keys), and that you want to sort in descending order. An example of the syntax is below:

.sortBy(***insert a lambda function here to show what you want to sort by***,ascending=False)

In [0]:
# YOUR CODE HERE

#### Practice exercise

In the below RDD I have loaded the shakespeare play Julius Caesar. Perform a wordcount to find the 10 most common words. You can insert more cells to work with if you need them.

In [0]:
jcRDD = sc.textFile("file:/databricks/driver/julius-caesar_TXT_FolgerShakespeare.txt")

In [0]:
#First we need to clean the dataset (split, lowercase, punctuation, and the lenght of the strings)

In [0]:
# Then, do the count and sort by

## Sentiment analysis

Text information can be broadly categorized into two main types: facts and opinions. Facts
are objective expressions about something. Opinions are usually subjective expressions that
describe people’s sentiments, appraisals, and feelings toward a subject or topic.
Sentiment analysis can be modeled as a classification problem:
Classifying a sentence as expressing a positive, negative or neutral opinion, known as
polarity classification.

In an opinion, the entity the text talks about can be an object, its components, its aspects, its
attributes, or its features. It could also be a product, a service, an individual, an organization,
an event, or a topic. As an example, take a look at the opinion below:

"The battery life of this camera is too short."

A negative opinion is expressed about a feature (battery life) of an entity (camera).
A basic example of a rule-based implementation would be the following:
Define two lists of polarized words (e.g. negative words such as bad, worst, ugly, etc and
positive words such as good, best, beautiful, etc).
Given a text:

● Count the number of positive words that appear in the text.

● Count the number of negative words that appear in the text.

● If the number of positive word appearances is greater than the number of negative
word appearances return a positive sentiment, conversely, return a negative
sentiment. Otherwise, return neutral.

This system is very naïve since it doesn't take into account how words are combined in a
sequence or the strength of the positive or negative sentiment of the words. Instead we often
use a dictionary with words scored on a scale of -5 to 5 with more negative values
representing more negative words. We then look at the total or average score of all the
words in a text to determine if it is positive or negative on average.

Run the below cells to download a dictionary of words with their rating, higher ratings mean more positive words, negative rating means words with negative sentiment.

In [0]:
%sh wget https://gist.githubusercontent.com/damianesteban/06e8be3225f641100126/raw/a51c27d4e9cc242f829d895e23b4435021ab55e5/afinn-111.txt

In [0]:
#Tip: use split("\t") to split the text file using tab
ratingRDD  = sc.textFile("file:/databricks/driver/afinn-111.txt").map(lambda x: x.split("\t")).map(lambda x: (x[0], int(x[1])))
ratingRDD.take(10)

You can see we have a list of words with different ratings in a pairedRDD. In case you are interested here is an online tool that lets you test the sentiment of a word according to this dictionary:

https://darenr.github.io/afinn/

We can now join our dictionary with the works of shakespare wordcount and count up the total negative and positive words:

In [0]:
joinedRDD = reducedWordCountRDD.join(ratingRDD)
joinedRDD.take(10)

You should see we now have a list of words from Shakespeare that also exist in our rating dictionary, we also have the number of times they occur and their rating. We can then combine the rating and number of times the words occur, and add up all the values to see if the overall sentiment of shakespear is positive or negative. See if you can understand how we are doing this with map and reduce below.

In [0]:
wordSentimentRDD = joinedRDD.map(lambda x: (x[0], x[1][0] * x[1][1]))
wordSentimentRDD.take(10)

In [0]:
wordSentimentRDD.reduce(lambda x, y: ("Total Sentiment", x[1]+ y[1]))

The sum of total sentiment is postive, so accoring to our analysis the works of Shakespeare contain a positive sentiment in total.

#### Practice exercise

Can you find if the total sentiment of the play "Julius Caesar" is positive or negative? Remember you have already performed a wordcount on this play.

In [0]:
#First we need to join the RDD with the rating RDD

#Then multiply

#Finally sum up

## Bonus exercise
### Collocations
Co-occurrence (or collocation) analysis is a simple and popular method in digital and computational humanities for measuring associations between actors, entities, and concepts using large collections of texts. For a few examples of how co-occurrence analysis has been used recently in humanistic scholarship, check out these research articles from Literary and Linguistic Computing.

Weingart, Scott, and Jeana Jorgensen. 2013. “Computational Analysis of the Body in European Fairy Tales.” Literary and Linguistic Computing 28 (3): 404–16. doi:10.1093/llc/fqs015.

This study was a collaboration between a digital historian and a gender and folklore studies scholar. They asked whether European fairy tales construct and represent bodies differently according to gender. They used a collection of 233 fairy tales, and tagged passages based on whether they referred to men or women, and whether those characters were young or old. They then looked for co-occurrences of body terms (head, heart, hands, beard) and adjectives, and looked for clusters of co-occurring terms that correspond disproportionately to gender or age.

Pumfrey, Stephen, Paul Rayson, and John Mariani. 2012. “Experiments in 17th Century English: Manual versus Automatic Conceptual History.” Literary and Linguistic Computing 27 (4): 395–408. doi:10.1093/llc/fqs017.

These authors used their own concordance program to study changes over time in usage of the term "experiment" and "experimental", based on the co-occurrence of those terms with other scientific and religious terms.

Kimura, Fuminori, Takahiko Osaki, Taro Tezuka, and Akira Maeda. 2013. “Visualization of Relationships among Historical Persons from Japanese Historical Documents.” Literary and Linguistic Computing 28 (2): 271–78. doi:10.1093/llc/fqs045.

The Hōgen Rebellion (1156) in Japan was a roughly two-week conflict between factions of former Emperor Sutoku and Emperor Goshirakawa over a dispute about Imperial succession, and about the degree of influence of the aristocratic Fujiwara clan that had heavily ingratiated the Imperial family. This was seen as an important factor in the transition from Imperial to samurai-led governance in Japan. In this study, the authors asked whether they could use computational methods to infer associations among aristocrats or samurai belonging to each of the two factions supporting Emperor Sutoku and Emperor Goshirakawa. They used a set of diaries called the "Hyohanki," written by an aristocrat named Nobunori Taira between 1112 and 1187, which is considered to be a valuable source of information about the Hōgen Rebellion and surrounding events. Given a list of 78 people, and a list of Japanese place-names, they looked for co-occurrences of those people and places in specific diary entries, and used those co-occurrences to infer latent relationships among people based on their spatial activities. The assumption here is that if two people are operating in the same places, at around the same times, then they are more likely to have interacted with each other.

Here we will use spark to perform collocation analysis.
### Spark practice task
Here we will practive using spark for text analysis:

Start with the RDD shakespearRDD, remember in this RDD each line from the text is a row. Note these rows are stored as unicode, it will be useful to use transformation to create a new RDD which converts each line into word pairs. For example the following line "Hello, my name is Bob." should be converted to the following rows:

"hello_my"

"my_name"

"name_is"

"is_bob"

These pairs of words are sometimes called "bigrams". Note how we have: removed punctuation, converted to lower case, seperated words with a ' _ ' to create a bigram.

Hint: An easy way to do this is to define a new method in python, apply your method to the "Hello, my name is Bob." sentence and check you get the correct result (as above). You can then use the map function to apply this to the shakepeare RDD.

For example your function should work like this:

convertToWordPairs('Hello, my name is Bob.')

Out[68]: ['hello_my', 'my_name', 'name_is', 'is_bob']

In [0]:
import string
def convertToWordPairs(line):
    pairs = []
    # Clean the data by removing punctuation, transforming strings to lower case and then split out the words
    words = str(line).translate(str.maketrans('', '', string.punctuation)).lower().split(" ")
    # For each word (except the last) add that word and the word that follows as a bigram to the pair list
    for i in range(0, len(words) -1):
        # clean out any "words" with zero length
        if (len(words[i]) > 0 and len(words[i+1]) > 0 ):
            pairs.append('{}_{}'.format(words[i], words[i+1]))
    return pairs

In [0]:
#run this cell to check you get the right output. The four bigrams shown above.
convertToWordPairs('Hello, my name is Bob.')

Next you will want to apply your function to the text in such a way that we can create a RDD of key value pairs, where the Key is the bigram and the value is 1. We can these use the reduceByKey function to count the bigrams and create an RDD with bigram as the key and total count as the value.

Your final output should look something like this (these results are unsorted):

Out[72]: [('a_midsummer', 2),
 ('queen_of', 96),
 ('snug_joiner', 1),
 ('other_fairies', 3),
 ('youth_to', 15),
 ('hath_my', 49),
 ('rings_gauds', 1),
 ('within_his', 22),
 ('kind_wanting', 2),
 ('well_your', 13)]

In [0]:
#First create de words pairs

In [0]:
#And count

Find the 10 most common bigrams.

One option here would to be use the sortByKey function, however, this sorts by keys not values so you would need to figure out how to use it to sort by values. Alternatively there is a sort by function.

The most common should be "i_am" with 1830 occurances.

In [0]:
# Your code here